### 프로젝트 요구 추가 외부 라이브러리 설치

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install plotly
!pip install statsmodels
!pip install tensorflow
!pip install keras
!pip install scikit-learn

# 2013~2021년도 가뭄지수 종합

## 필요한 모듈 및 데이터 import

In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
from datetime import date, timedelta

In [ ]:
path = '../data/가뭄/원본/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]
edi_concat = pd.DataFrame()

for i in file_list_py:
    data = pd.read_csv(path + i)
    edi_concat = pd.concat([edi_concat, data])
    
edi_concat = edi_concat.reset_index(drop = True)

### 지점명, 시군명 변경

In [ ]:
#기상 관측 지점 데이터
ob_point = pd.read_csv('../data/지역/관측지점_원본_기상청.csv')
ob_point

In [ ]:
ob_point.drop(columns = ['종료시각', '시작시각', '노장 해발고도(m)','기압계', '기온계', '풍속계', '강우계', '위도', '경도'],
             inplace = True)
ob_point.rename(columns = {'(현)지점명':'지점명', '(현)주소':'주소'}, inplace = True)

In [ ]:
ob_point.dropna(inplace = True)

In [ ]:
ob_point.주소 = ob_point.주소.str.split(" ")
region = []
for item in ob_point['주소']:
    first = item[0]
    second = item[1]
    print(first)
    print(second)
    if "시" in second:
        region.append(second)
    elif "군" in second:
        region.append(second)
    elif "시" in first:
        region.append(first)
ob_point["주소"] = region

ob_point.drop(columns='지점명',inplace = True)
ob_point.지점코드 = ob_point.지점코드.astype(int)

# 특이하게 저장된 부분들 편집
ob_point.replace('포항시남구',"포항시", inplace=True)
ob_point.replace('전주시덕진구',"전주시", inplace=True)
ob_point.replace('청주시흥덕구',"청주시", inplace=True)
ob_point.replace('\xa0서울특별시', '서울특별시', inplace = True)
ob_point.replace('\xa0인천광역시', '인천광역시', inplace = True)
ob_point.replace('\xa0대전광역시', '대전광역시', inplace = True)
ob_point.replace('\xa0대구광역시', '대구광역시', inplace = True)
ob_point.replace('\xa0울산광역시', '울산광역시', inplace = True)
ob_point.replace('\xa0광주광역시', '광주광역시', inplace = True)
ob_point.replace('\xa0부산광역시', '부산광역시', inplace = True)

# edi 데이터에 주소 정보를 결합하기 위해 사전으로 변경
point_dict = ob_point.set_index('지점코드').T.to_dict('records')
point_dict = point_dict[0]

In [ ]:
edi_concat['시군명'] = edi_concat.지점코드.apply(lambda x: point_dict[x])

In [ ]:
edi_concat.head()

In [ ]:
edi_concat.rename(columns = {'spi1':'SPI1', 'spi3':'SPI3', 'edi':'EDI'}, inplace = True)
edi_concat = edi_concat[['발표일자', '지점명', '시군명', 'SPI1', 'SPI3', 'EDI']]
edi_concat

In [ ]:
# 문자형 결측치 NaN으로 변형 후 자료형 변경
edi_concat.replace('-', np.nan, inplace = True)
edi_concat.SPI1 = edi_concat.SPI1.astype(float)
edi_concat.SPI3 = edi_concat.SPI3.astype(float)
edi_concat.EDI = edi_concat.EDI.astype(float)
edi_concat.info()

In [ ]:
#날짜 중에 불연속 지점 탐색 #해당 날짜 데이터 없음
start_date = edi_concat['발표일자'].unique()[0]
td = timedelta(days = 1)

for dt in edi_concat['발표일자'].unique()[1:]:
    if dt != date.isoformat(date.fromisoformat(start_date) + td) :
        print(date.fromisoformat(start_date) + td)
        #print(start_date, dt)
    start_date = dt

In [ ]:
edi_concat.info()

In [ ]:
edi_concat = edi_concat[['발표일자', '지점명', '시군명', 'SPI1', 'SPI3', 'EDI']]

### 데이터셋 파일 저장

In [ ]:
edi_concat.to_csv('../data/가뭄/2013_2020_가뭄지수데이터.csv', encoding = 'utf-8-sig')